In [1]:
# %pip install openai>1.50.0 langchain>0.3.0 langgraph langchainhub langchain-openai langchain-community langchain-cli langchain_ollama tavily-python>=0.5.0 langchain_nomic nomic[local] langserve faiss-cpu tiktoken pypdf chroma jira google-search-results numexpr beautifulsoup4 scikit-learn

In [2]:
# uncomment this if running locally
# from dotenv import load_dotenv

# load_dotenv()

# Or if you are in Colab, uncoment below and add your api key
import os
os.environ["OPENAI_API_KEY"] = "your-api-key"

# Common Sense Agent

1. Model (large language model like ChatGPT, or Claude)
2. Prompt (system message and the user input)
3. Tool calling

In [2]:
from langchain_openai import ChatOpenAI
import os
# os.environ["OPENAI_API_KEY"] = "your openai api key"
llm = ChatOpenAI(model="gpt-4o", temperature=0)

llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d8864f8b6b', 'id': 'chatcmpl-BNMJeyOWphQc0bAPnWcsWhguLsduP', 'finish_reason': 'stop', 'logprobs': None}, id='run-0258923f-2eec-4932-a278-9ccb94829779-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [3]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Answer this question: {question} in bullet points. Output:\n")

In [4]:
chain = prompt | llm

chain

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Answer this question: {question} in bullet points. Output:\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10f018490>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10f021710>, root_client=<openai.OpenAI object at 0x10e9dda90>, root_async_client=<openai.AsyncOpenAI object at 0x10f018650>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.openai.com/v1')

In [5]:
output = chain.invoke("Write down names of cats.")
output

AIMessage(content='- Whiskers\n- Mittens\n- Shadow\n- Luna\n- Oliver\n- Bella\n- Simba\n- Chloe\n- Max\n- Tiger', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 23, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6b6e24b474', 'id': 'chatcmpl-BNMJoN4Mjz1gVwRPwSIaaBFof6TQ9', 'finish_reason': 'stop', 'logprobs': None}, id='run-7ddd3129-dd9d-43f8-a619-3116d3562e32-0', usage_metadata={'input_tokens': 23, 'output_tokens': 33, 'total_tokens': 56, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
from IPython.display import Markdown

Markdown(output.content)

- Whiskers
- Mittens
- Shadow
- Luna
- Oliver
- Bella
- Simba
- Chloe
- Max
- Tiger

In [7]:
# You could use TavilySearchResults as well for this same example!
from langchain_core.tools import tool
from serpapi import GoogleSearch

serpapi_params = {
    "engine": "google",
    "api_key": os.environ["SERPAPI_KEY"]
}

query = "Cool buildings in Paris"
search = GoogleSearch({**serpapi_params, "q":query, "n": 3})
search.get_dict()["organic_results"]

[{'position': 1,
  'title': '25 Must-See Paris Landmarks',
  'link': 'https://www.architecturaldigest.com/gallery/paris-architectural-landmarks',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.architecturaldigest.com/gallery/paris-architectural-landmarks&ved=2ahUKEwjrzpjCvd-MAxUUEFkFHdY-EVsQFnoECBwQAQ',
  'displayed_link': 'https://www.architecturaldigest.com › ... › travel',
  'thumbnail': 'https://serpapi.com/searches/68012b60dda8681ef44c5f13/images/b82971cf6568ad27e2fa13ba6a1ef50fcedef7e58fe760b01529225406d3a2a2.jpeg',
  'favicon': 'https://serpapi.com/searches/68012b60dda8681ef44c5f13/images/b82971cf6568ad27e2fa13ba6a1ef50f8f513dcefb7186a53f3be813729d0233.png',
  'date': 'Feb 18, 2016',
  'snippet': 'The Eiffel Tower shoots up from one end. The gilded cap of the Les Invalides gleams just a few blocks away. The Louvre stretches across its ...',
  'snippet_highlighted_words': ['The Eiffel Tower shoots up from one end'],
  'source': '

In [8]:
results = search.get_dict()["organic_results"]
contexts = "\n---\n".join(
       ["\n".join([x["title"], x["snippet"], x["link"]]) for x in results]
   )
contexts

"25 Must-See Paris Landmarks\nThe Eiffel Tower shoots up from one end. The gilded cap of the Les Invalides gleams just a few blocks away. The Louvre stretches across its ...\nhttps://www.architecturaldigest.com/gallery/paris-architectural-landmarks\n---\n8 beautiful buildings in Paris\nA selection of the most beautiful buildings in Paris: Galeries Lafayette, Palais Garnier, Galerie Vivienne and more amazing architecture.\nhttps://brightnomad.net/beautiful-buildings-in-paris/\n---\nGoing to Paris soon, which buildings should I visit?\nYou can visit la maison la roche by le corbusier, le pavillon de l'arsenal (architecture and urban planning museum), Beaubourg, la fondation ...\nhttps://www.reddit.com/r/architecture/comments/19830tq/going_to_paris_soon_which_buildings_should_i_visit/\n---\nTHE 15 BEST Paris Architectural Buildings (2025)\n1. Louvre Museum ... Whether you're an art enthusiast or a history buff, the Louvre is a must-visit destination that will leave you in awe.\nhttps://ww

In [9]:
@tool
def web_search(query: str) -> str:
    """
    Finds general knowledge information using Google search. Can also be used
    to augment more 'general' knowledge to a previous specialist query.
    """
    search = GoogleSearch({**serpapi_params, "q":query, "n": 3})
    results = search.get_dict()["organic_results"]
    contexts = "\n---\n".join(
         ["\n".join([x["title"], x["snippet"], x["link"]]) for x in results]
        )
    
    return contexts

web_search.invoke("Cool buildings in Paris")

"25 Must-See Paris Landmarks\nThe Eiffel Tower shoots up from one end. The gilded cap of the Les Invalides gleams just a few blocks away. The Louvre stretches across its ...\nhttps://www.architecturaldigest.com/gallery/paris-architectural-landmarks\n---\n8 beautiful buildings in Paris\nA selection of the most beautiful buildings in Paris: Galeries Lafayette, Palais Garnier, Galerie Vivienne and more amazing architecture.\nhttps://brightnomad.net/beautiful-buildings-in-paris/\n---\nGoing to Paris soon, which buildings should I visit?\nYou can visit la maison la roche by le corbusier, le pavillon de l'arsenal (architecture and urban planning museum), Beaubourg, la fondation ...\nhttps://www.reddit.com/r/architecture/comments/19830tq/going_to_paris_soon_which_buildings_should_i_visit/\n---\nTHE 15 BEST Paris Architectural Buildings (2025)\n1. Louvre Museum ... Whether you're an art enthusiast or a history buff, the Louvre is a must-visit destination that will leave you in awe.\nhttps://ww

In [11]:
llm_with_tool = llm.bind_tools([web_search])

research_chain = prompt | llm_with_tool 

output_research_chain = research_chain.invoke("What are the cool buildings in Paris?")

In [14]:
output_research_chain.tool_calls

[{'name': 'web_search',
  'args': {'query': 'cool buildings in Paris'},
  'id': 'call_4Z1kuzkJmr4zWqQ6yzXNf1GD',
  'type': 'tool_call'}]

In [11]:
sample_search_result = research_chain.invoke("What are the cool buildings in Paris?")
sample_search_result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_y6OYjPZ6KDZn1kd6ZExc9bgY', 'function': {'arguments': '{"query":"cool buildings in Paris"}', 'name': 'web_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 82, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-286bbd45-e787-4447-b128-19d7489a9bf2-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'cool buildings in Paris'}, 'id': 'call_y6OYjPZ6KDZn1kd6ZExc9bgY', 'type': 'tool_call'}], usage_metadata={'input_tokens': 82, 'output_tokens': 18, 'total_tokens': 100, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_detail

In [12]:
sample_search_result.tool_calls

[{'name': 'web_search',
  'args': {'query': 'cool buildings in Paris'},
  'id': 'call_y6OYjPZ6KDZn1kd6ZExc9bgY',
  'type': 'tool_call'}]

In [13]:
sample_search_result.tool_calls[0]['args']

{'query': 'cool buildings in Paris'}

In [15]:
research_chain = prompt | llm_with_tool | (lambda x: x.tool_calls[0]['args'])

research_chain.invoke("What are the cool buildings in Paris?")

{'query': 'cool buildings in Paris'}

In [21]:
output_of_llm_with_tool = llm_with_tool.invoke("What are the cool buildings in Paris?")

In [22]:
extract_tool_call = lambda x: x.tool_calls[0]['args']

extract_tool_call(output_of_llm_with_tool)

{'query': 'cool buildings in Paris'}

In [23]:
web_search.invoke(extract_tool_call(output_of_llm_with_tool))

"25 Must-See Paris Landmarks\nThe Eiffel Tower shoots up from one end. The gilded cap of the Les Invalides gleams just a few blocks away. The Louvre stretches across its ...\nhttps://www.architecturaldigest.com/gallery/paris-architectural-landmarks\n---\n8 beautiful buildings in Paris\nA selection of the most beautiful buildings in Paris: Galeries Lafayette, Palais Garnier, Galerie Vivienne and more amazing architecture.\nhttps://brightnomad.net/beautiful-buildings-in-paris/\n---\nGoing to Paris soon, which buildings should I visit?\nYou can visit la maison la roche by le corbusier, le pavillon de l'arsenal (architecture and urban planning museum), Beaubourg, la fondation ...\nhttps://www.reddit.com/r/architecture/comments/19830tq/going_to_paris_soon_which_buildings_should_i_visit/\n---\nTHE 15 BEST Paris Architectural Buildings (2025)\n1. Louvre Museum ... Whether you're an art enthusiast or a history buff, the Louvre is a must-visit destination that will leave you in awe.\nhttps://ww

In [16]:
research_chain = prompt | llm_with_tool | (lambda x: x.tool_calls[0]['args']) | web_search

research_chain.invoke("What are the cool buildings in Paris?")

"25 Must-See Paris Landmarks\nThe Eiffel Tower shoots up from one end. The gilded cap of the Les Invalides gleams just a few blocks away. The Louvre stretches across its ...\nhttps://www.architecturaldigest.com/gallery/paris-architectural-landmarks\n---\n8 beautiful buildings in Paris\nA selection of the most beautiful buildings in Paris: Galeries Lafayette, Palais Garnier, Galerie Vivienne and more amazing architecture.\nhttps://brightnomad.net/beautiful-buildings-in-paris/\n---\nGoing to Paris soon, which buildings should I visit?\nYou can visit la maison la roche by le corbusier, le pavillon de l'arsenal (architecture and urban planning museum), Beaubourg, la fondation ...\nhttps://www.reddit.com/r/architecture/comments/19830tq/going_to_paris_soon_which_buildings_should_i_visit/\n---\nTHE 15 BEST Paris Architectural Buildings (2025)\n1. Louvre Museum ... Whether you're an art enthusiast or a history buff, the Louvre is a must-visit destination that will leave you in awe.\nhttps://ww

In [17]:
def fake_agent(query: str) -> str:
    search_result = research_chain.invoke("What are the cool buildings in Paris?")
    fake_agent = llm.invoke(f"The user query was: {query}. The web search result for\
        that is: {search_result}. Give a summarized answer:")
    
    return fake_agent.content

fake_agent("What are the cool buildings in Paris?")

'Paris is home to a stunning array of architectural marvels, both historic and contemporary. Some of the must-see buildings include:\n\n1. **Eiffel Tower** - The iconic symbol of Paris.\n2. **Louvre Museum** - A historic and vast museum with stunning architecture.\n3. **Notre-Dame de Paris** - A masterpiece of Gothic architecture.\n4. **Centre Georges Pompidou** - Known for its modern and innovative design.\n5. **Fondation Louis Vuitton** - A contemporary art museum with striking architecture.\n6. **Palais Garnier** - A grand opera house with opulent interiors.\n7. **Sainte Chapelle** - Famous for its stunning stained glass windows.\n8. **Arc de Triomphe** - A monumental arch honoring those who fought for France.\n9. **Les Invalides** - Known for its gilded dome and military history.\n10. **Galeries Lafayette** - A department store with a beautiful glass dome.\n\nThese buildings showcase the rich architectural diversity and history of Paris, making them essential stops for any visitor.

In [18]:
from langchain_core.runnables import RunnablePassthrough
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
    ("system", "You are a helpful assistant"),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
    ]
                                          )

llm = ChatOpenAI(model="gpt-4o", temperature=0)

tools = [web_search]

llm_with_tools = llm.bind_tools(tools)

research_agent = (
                 RunnablePassthrough.assign(
                    agent_scratchpad=lambda x: format_to_tool_messages(x['intermediate_steps'])
                    )
) | prompt | llm_with_tools | ToolsAgentOutputParser()

research_agent.invoke({"input": "What are the cool buildings in Paris?", "intermediate_steps": []})

[ToolAgentAction(tool='web_search', tool_input={'query': 'cool buildings in Paris'}, log="\nInvoking: `web_search` with `{'query': 'cool buildings in Paris'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WCJOtK1AGRhVPGmEDNVxkNN1', 'function': {'arguments': '{"query":"cool buildings in Paris"}', 'name': 'web_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 78, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a6dfaf2bcf', 'id': 'chatcmpl-BNMOwVlfIIa21lRmwOgXBViinbZ1y', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-13c66a05-25c9-468b-b618-b7ad392984a7-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'cool buildings

In [19]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=research_agent,tools=tools)
agent_executor.invoke({'input': 'What are cool buildings in Alicante Spain?'})

{'input': 'What are cool buildings in Alicante Spain?',
 'output': "Alicante, Spain, is home to several cool and iconic buildings. Here are some notable ones:\n\n1. **Castillo de Santa Bárbara**: A medieval fortress perched on a hill overlooking the city, offering stunning views and a glimpse into Alicante's history.\n\n2. **Rellotge de Sol de la Basilica de Santa Maria**: Known for its architectural beauty and historical significance.\n\n3. **Oficina De Correos**: The main post office building, known for its architectural design.\n\n4. **Sarrio Tower**: A modern architectural landmark in the city.\n\n5. **Archives Municipales**: A building that holds historical records and is noted for its architecture.\n\n6. **46 Calle San Fernando Building**: A notable building in Alicante for its design.\n\n7. **Casa De Bruixes Building**: Known for its unique architectural style.\n\n8. **Comandancia Naval Alacant**: A naval command building with historical and architectural interest.\n\nThese buil

Wwe can automate part of this process by using a built in method in LangChain
that abstracts away the chaining procedure.

In [20]:
from langchain.agents import create_tool_calling_agent
from langchain_openai import ChatOpenAI
from langchain import hub

llm = ChatOpenAI(model="gpt-4o", temperature=0)

tools = [web_search]

prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({'input': 'What are cool buildings in Alicante Spain?'})

{'input': 'What are cool buildings in Alicante Spain?',
 'output': "Alicante, Spain, is home to several cool and iconic buildings. Here are some notable ones:\n\n1. **Castle of Santa Barbara**: A medieval fortress perched on a hill overlooking the city, offering stunning views and a glimpse into Alicante's history.\n\n2. **Casa Carbonell**: Built between 1922 and 1925, this iconic building is located at the end of the Explanada de España and is a prime example of early 20th-century architecture.\n\n3. **Oficina De Correos**: The main post office building, known for its architectural beauty.\n\n4. **Sarrio Tower**: A modern architectural landmark in the city.\n\n5. **Comandancia Naval Alacant Building**: Another significant building with unique architectural features.\n\n6. **Casa De Bruixes**: Known for its distinctive design and historical significance.\n\nThese buildings reflect a mix of historical and modern architectural styles, making Alicante a fascinating city for architecture e

The new way of doing it!

# JJ's question: 
How does the observation know when tools generate their output? Is it a synchro calls or asynchro functional calls?

# PS's question: 
can u explain lamda ...what it does ...

# AS's question: 
In Python, a lambda function is an anonymous function defined using the lambda keyword. It's a concise way to create small, one-line functions, often used as arguments to other functions or within operations that require a simple function. Lambda functions can take any number of arguments but can only have one expression, which is automatically returned.

In [21]:
# No system message in this simple example
from langgraph.prebuilt import create_react_agent

langgraph_agent_executor = create_react_agent(llm, tools)

query = 'What are cool buildings in Alicante Spain?'

messages = langgraph_agent_executor.invoke({"messages": [("human", query)]})
{
    "input": query,
    "output": messages["messages"][-1].content,
}
messages

{'messages': [HumanMessage(content='What are cool buildings in Alicante Spain?', additional_kwargs={}, response_metadata={}, id='c52ef873-d890-4761-82ec-b1d4e279f297'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hsiTEtEKggKtrCRtIATpwTz4', 'function': {'arguments': '{"query":"cool buildings in Alicante Spain"}', 'name': 'web_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 72, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f52a4e87-5a6a-4209-8726-88f97c47f7e6-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'cool buildings in Alicante Spain'}, 'id': 'call_hsiTEtEKggKtrC

In [22]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import create_react_agent

system_message = "You are a helpful assistant. Respond only in Spanish."
# This could also be a SystemMessage object
# system_message = SystemMessage(content="You are a helpful assistant. Respond only in Spanish.")

langgraph_agent_executor = create_react_agent(llm, tools, prompt=system_message)


messages = langgraph_agent_executor.invoke({"messages": [("user", query)]})
messages

{'messages': [HumanMessage(content='What are cool buildings in Alicante Spain?', additional_kwargs={}, response_metadata={}, id='1ba8c792-fb63-45c0-b213-f8b4fdfaff63'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LMQElH56T0hXFrI6Ubxzj9Iw', 'function': {'arguments': '{"query":"cool buildings in Alicante Spain"}', 'name': 'web_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 83, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-60bd3b56-2c70-4153-85ac-1cb3f817183b-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'cool buildings in Alicante Spain'}, 'id': 'call_LMQElH56T0hXF

In [24]:
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Respond only in Spanish."),
        ("placeholder", "{messages}"),
        ("user", "Also say 'Pandamonium!' after the answer."),
    ]
)

# alternatively, this can be passed as a function, e.g.
# def prompt(state: AgentState):
#     return (
#         [SystemMessage(content="You are a helpful assistant. Respond only in Spanish.")] +
#         state["messages"] +
#         [HumanMessage(content="Also say 'Pandamonium!' after the answer.")]
#     )


langgraph_agent_executor = create_react_agent(llm, tools, prompt=prompt)


messages = langgraph_agent_executor.invoke({"messages": [("human", query)]})

In [25]:
print(
    {
        "input": query,
        "output": messages["messages"][-1].content,
    }
)

{'input': 'Cool buildings in Paris', 'output': 'Aquí tienes una lista de algunos edificios impresionantes en París:\n\n1. **Torre Eiffel**: Un ícono mundialmente reconocido de la arquitectura parisina.\n2. **Museo del Louvre**: No solo es famoso por su arte, sino también por su impresionante arquitectura.\n3. **Catedral de Notre-Dame**: Un ejemplo magnífico de la arquitectura gótica.\n4. **Centro Pompidou**: Conocido por su diseño moderno y su estructura expuesta.\n5. **Palais Garnier**: La opulenta ópera de París, famosa por su arquitectura neobarroca.\n6. **Fundación Louis Vuitton**: Un edificio contemporáneo diseñado por Frank Gehry.\n7. **Sainte-Chapelle**: Famosa por sus impresionantes vitrales.\n8. **Galeries Lafayette**: Con su cúpula de vidrio y hierro, es un ejemplo de la arquitectura Art Nouveau.\n9. **Palais de Tokyo**: Un centro de arte contemporáneo con una arquitectura única.\n10. **La Défense**: El distrito de negocios con rascacielos modernos y el famoso Arco de la Défe

# Memory in Legacy LangChain
source: [LangChain Docs](https://python.langchain.com/docs/how_to/migrate_agent/)

In [24]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")
memory = InMemoryChatMessageHistory(session_id="test-session")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        # First put the history
        ("placeholder", "{chat_history}"),
        # Then the new input
        ("human", "{input}"),
        # Finally the scratchpad
        ("placeholder", "{agent_scratchpad}"),
    ]
)


@tool
def magic_function(input: int) -> int:
    """Applies a magic function to an input."""
    return input + 2


tools = [magic_function]


agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

config = {"configurable": {"session_id": "test-session"}}
print(
    agent_with_chat_history.invoke(
        {"input": "Hi, I'm polly! What's the output of magic_function of 3?"}, config
    )["output"]
)
print("---")
print(agent_with_chat_history.invoke({"input": "Remember my name?"}, config)["output"])
print("---")
print(
    agent_with_chat_history.invoke({"input": "what was that output again?"}, config)[
        "output"
    ]
)

The output of `magic_function` for the input 3 is 5.
---
Yes, you mentioned that your name is Polly.
---
The output of `magic_function` for the input 3 is 5.


In the new LangGraph,
memory = [checkpointing](https://langchain-ai.github.io/langgraph/reference/checkpoints/).

In [ ]:
from langgraph.checkpoint.memory import MemorySaver  # an in-memory checkpointer
from langgraph.prebuilt import create_react_agent

system_message = "You are a helpful assistant."
# This could also be a SystemMessage object
# system_message = SystemMessage(content="You are a helpful assistant. Respond only in Spanish.")

memory = MemorySaver()
langgraph_agent_executor = create_react_agent(
    model, tools, prompt=system_message, checkpointer=memory
)

config = {"configurable": {"thread_id": "test-thread"}}
print(
    langgraph_agent_executor.invoke(
        {
            "messages": [
                ("user", "Hi, I'm polly! What's the output of magic_function of 3?")
            ]
        },
        config,
    )["messages"][-1].content
)
print("---")
print(
    langgraph_agent_executor.invoke(
        {"messages": [("user", "Remember my name?")]}, config
    )["messages"][-1].content
)
print("---")
print(
    langgraph_agent_executor.invoke(
        {"messages": [("user", "what was that output again?")]}, config
    )["messages"][-1].content
)